In [1]:
import os
import itertools
import pandas as pd
import numpy as np

In [2]:
def format_time(seconds):
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    seconds = seconds % 60
    return f"{hours} hrs. {minutes} mins. {seconds} secs."

In [3]:
alphas = [0.01, 0.001, 0.0001, 1e-05, 1e-06, 0.0]
criterion = 'linear_probing'
dataset_directory = '/cluster/tufts/hugheslab/eharve06/CIFAR-10'
experiments_directory = '/cluster/tufts/hugheslab/eharve06/data-emphasized-ELBo/experiments/tuned_CIFAR-10'
lr_0s = [0.1, 0.01, 0.001, 0.0001]
ns = [100, 1000, 10000, 50000]
random_states = [1001, 2001, 3001]

columns = ['criterion', 'model_name', 'n', 'random_state', 'runtime', 'val_acc', 'val_nll']
tuned_df = pd.DataFrame(columns=columns)

for alpha, lr_0, n, random_state in itertools.product(alphas, lr_0s, ns, random_states):
    model_name = f'{criterion}_alpha={alpha}_lr_0={lr_0}_n={n}_random_state={random_state}'
    temp_df = pd.read_csv(f'{experiments_directory}/{model_name}.csv')
    row = [criterion, model_name, n, random_state, temp_df['train_sec/epoch'].sum(), temp_df.val_or_test_acc.values[-1], temp_df.val_or_test_nll.values[-1]]
    tuned_df.loc[len(tuned_df)] = row
    
min_indices = tuned_df.groupby(['criterion', 'n', 'random_state'])['val_nll'].idxmin()
    
experiments_directory = '/cluster/tufts/hugheslab/eharve06/data-emphasized-ELBo/experiments/retrained_CIFAR-10'

columns = ['criterion', 'model_name', 'n', 'random_state', 'runtime', 'test_acc', 'test_nll']
retrained_df = pd.DataFrame(columns=columns)

for index, row in tuned_df.loc[min_indices].iterrows():
    temp_df = pd.read_csv(f'{experiments_directory}/{row.model_name}.csv')
    row = [row.criterion, row.model_name, row.n, row.random_state, temp_df['train_sec/epoch'].sum(), temp_df.val_or_test_acc.values[-1], temp_df.val_or_test_nll.values[-1]]
    retrained_df.loc[len(retrained_df)] = row

retrained_df

,criterion,model_name,n,random_state,runtime,test_acc,test_nll
0,linear_probing,linear_probing_alpha=1e-06_lr_0=0.0001_n=100_r...,100,1001,1294.746890,0.6123,1.196466
1,linear_probing,linear_probing_alpha=0.01_lr_0=0.0001_n=100_ra...,100,2001,1246.488588,0.5927,1.246481
2,linear_probing,linear_probing_alpha=0.01_lr_0=0.001_n=100_ran...,100,3001,1263.100174,0.5994,1.204519
3,linear_probing,linear_probing_alpha=0.01_lr_0=0.001_n=1000_ra...,1000,1001,1575.736796,0.7476,0.747005
4,linear_probing,linear_probing_alpha=0.01_lr_0=0.001_n=1000_ra...,1000,2001,1571.332820,0.7465,0.746704
5,linear_probing,linear_probing_alpha=0.01_lr_0=0.1_n=1000_rand...,1000,3001,1676.978448,0.7418,0.762468
6,linear_probing,linear_probing_alpha=0.001_lr_0=0.1_n=10000_ra...,10000,1001,1641.763996,0.8143,0.541357
7,linear_probing,linear_probing_alpha=0.001_lr_0=0.1_n=10000_ra...,10000,2001,1624.493030,0.8130,0.548010
8,linear_probing,linear_probing_alpha=0.001_lr_0=0.1_n=10000_ra...,10000,3001,1501.117146,0.8138,0.548427
9,linear_probing,linear_probing_alpha=0.0_lr_0=0.1_n=50000_rand...,50000,1001,1505.444547,0.8321,0.483906


In [4]:
condition = (tuned_df.n==50000)&(tuned_df.criterion=='linear_probing')&(tuned_df.random_state==1001)
temp_df = tuned_df.loc[condition]
total_time = temp_df.runtime.sum()
best_model_name = temp_df.loc[temp_df.val_nll.idxmin(), 'model_name']
total_time += retrained_df.loc[retrained_df.model_name==best_model_name, 'runtime'].item()
average_time = total_time/(temp_df.shape[0]+1)
print(f'Average runtime (sec.): {format_time(average_time)}')
print(f'Total time (sec.): {format_time(total_time)}')

Average runtime (sec.): 0.0 hrs. 25.0 mins. 40.31537203788753 secs.
Total time (sec.): 10.0 hrs. 41.0 mins. 47.88430094718933 secs.


In [5]:
alphas = [0.01, 0.001, 0.0001, 1e-05, 1e-06, 0.0]
betas = [0.01, 0.001, 0.0001, 1e-05, 1e-06, 0.0]
criterion = 'l2-sp'
dataset_directory = '/cluster/tufts/hugheslab/eharve06/CIFAR-10'
experiments_directory = '/cluster/tufts/hugheslab/eharve06/data-emphasized-ELBo/experiments/tuned_CIFAR-10'
lr_0s = [0.1, 0.01, 0.001, 0.0001]
ns = [100, 1000, 10000, 50000]
random_states = [1001, 2001, 3001]

columns = ['criterion', 'model_name', 'n', 'random_state', 'runtime', 'val_acc', 'val_nll']
tuned_df = pd.DataFrame(columns=columns)

for alpha, beta, lr_0, n, random_state in itertools.product(alphas, betas, lr_0s, ns, random_states):
    model_name = f'{criterion}_alpha={alpha}_beta={beta}_lr_0={lr_0}_n={n}_random_state={random_state}'
    temp_df = pd.read_csv(f'{experiments_directory}/{model_name}.csv')
    row = [criterion, model_name, n, random_state, temp_df['train_sec/epoch'].sum(), temp_df.val_or_test_acc.values[-1], temp_df.val_or_test_nll.values[-1]]
    tuned_df.loc[len(tuned_df)] = row
    
min_indices = tuned_df.groupby(['criterion', 'n', 'random_state'])['val_nll'].idxmin()
    
experiments_directory = '/cluster/tufts/hugheslab/eharve06/data-emphasized-ELBo/experiments/retrained_CIFAR-10'

columns = ['criterion', 'model_name', 'n', 'random_state', 'runtime', 'test_acc', 'test_nll']
retrained_df = pd.DataFrame(columns=columns)

for index, row in tuned_df.loc[min_indices].iterrows():
    temp_df = pd.read_csv(f'{experiments_directory}/{row.model_name}.csv')
    row = [row.criterion, row.model_name, row.n, row.random_state, temp_df['train_sec/epoch'].sum(), temp_df.val_or_test_acc.values[-1], temp_df.val_or_test_nll.values[-1]]
    retrained_df.loc[len(retrained_df)] = row

retrained_df

,criterion,model_name,n,random_state,runtime,test_acc,test_nll
0,l2-sp,l2-sp_alpha=0.001_beta=0.001_lr_0=0.001_n=100_...,100,1001,1742.681498,0.6891,0.936994
1,l2-sp,l2-sp_alpha=0.01_beta=0.01_lr_0=0.001_n=100_ra...,100,2001,1777.736538,0.6589,1.022275
2,l2-sp,l2-sp_alpha=0.01_beta=0.001_lr_0=0.1_n=100_ran...,100,3001,1829.155005,0.7228,0.853342
3,l2-sp,l2-sp_alpha=0.001_beta=0.0001_lr_0=0.1_n=1000_...,1000,1001,2130.297514,0.8939,0.410627
4,l2-sp,l2-sp_alpha=0.001_beta=0.01_lr_0=0.1_n=1000_ra...,1000,2001,2230.276774,0.8836,0.390025
5,l2-sp,l2-sp_alpha=0.01_beta=0.001_lr_0=0.1_n=1000_ra...,1000,3001,2272.242814,0.8830,0.396594
6,l2-sp,l2-sp_alpha=0.001_beta=0.001_lr_0=0.1_n=10000_...,10000,1001,2108.784144,0.9533,0.157821
7,l2-sp,l2-sp_alpha=0.001_beta=0.01_lr_0=0.1_n=10000_r...,10000,2001,2248.242524,0.9539,0.154652
8,l2-sp,l2-sp_alpha=0.001_beta=0.01_lr_0=0.1_n=10000_r...,10000,3001,2224.394013,0.9555,0.150124
9,l2-sp,l2-sp_alpha=0.001_beta=0.01_lr_0=0.1_n=50000_r...,50000,1001,2653.558301,0.9720,0.092658


In [6]:
condition = (tuned_df.n==50000)&(tuned_df.criterion=='l2-sp')&(tuned_df.random_state==1001)
temp_df = tuned_df.loc[condition]
total_time = temp_df.runtime.sum()
best_model_name = temp_df.loc[temp_df.val_nll.idxmin(), 'model_name']
total_time += retrained_df.loc[retrained_df.model_name==best_model_name, 'runtime'].item()
average_time = total_time/(temp_df.shape[0]+1)
print(f'Average runtime (sec.): {format_time(average_time)}')
print(f'Total time (sec.): {format_time(total_time)}')

Average runtime (sec.): 0.0 hrs. 36.0 mins. 36.78224851674031 secs.
Total time (sec.): 88.0 hrs. 28.0 mins. 53.426034927368164 secs.


In [7]:
betas = [0.01, 0.001, 0.0001, 1e-05, 1e-06, 0.0]
criterion = 'ptyl'
dataset_directory = '/cluster/tufts/hugheslab/eharve06/CIFAR-10'
experiments_directory = '/cluster/tufts/hugheslab/eharve06/data-emphasized-ELBo/experiments/tuned_CIFAR-10'
lambds = np.logspace(start=0, stop=9, num=10)
lr_0s = [0.1, 0.01, 0.001, 0.0001]
ns = [100, 1000, 10000, 50000]
random_states = [1001, 2001, 3001]

columns = ['criterion', 'model_name', 'n', 'random_state', 'runtime', 'val_acc', 'val_nll']
tuned_df = pd.DataFrame(columns=columns)

for beta, lambd, lr_0, n, random_state in itertools.product(betas, lambds, lr_0s, ns, random_states):
    model_name = f'{criterion}_beta={beta}_lambd={lambd}_lr_0={lr_0}_n={n}_random_state={random_state}'
    temp_df = pd.read_csv(f'{experiments_directory}/{model_name}.csv')
    row = [criterion, model_name, n, random_state, temp_df['train_sec/epoch'].sum(), temp_df.val_or_test_acc.values[-1], temp_df.val_or_test_nll.values[-1]]
    tuned_df.loc[len(tuned_df)] = row
    
min_indices = tuned_df.groupby(['criterion', 'n', 'random_state'])['val_nll'].idxmin()

experiments_directory = '/cluster/tufts/hugheslab/eharve06/data-emphasized-ELBo/experiments/retrained_CIFAR-10'

columns = ['criterion', 'model_name', 'n', 'random_state', 'runtime', 'test_acc', 'test_nll']
retrained_df = pd.DataFrame(columns=columns)

for index, row in tuned_df.loc[min_indices].iterrows():
    temp_df = pd.read_csv(f'{experiments_directory}/{row.model_name}.csv')
    row = [row.criterion, row.model_name, row.n, row.random_state, temp_df['train_sec/epoch'].sum(), temp_df.val_or_test_acc.values[-1], temp_df.val_or_test_nll.values[-1]]
    retrained_df.loc[len(retrained_df)] = row

retrained_df

,criterion,model_name,n,random_state,runtime,test_acc,test_nll
0,ptyl,ptyl_beta=0.0_lambd=10.0_lr_0=0.001_n=100_rand...,100,1001,1721.813070,0.6920,0.935029
1,ptyl,ptyl_beta=0.01_lambd=1.0_lr_0=0.01_n=100_rando...,100,2001,1752.895676,0.6960,0.912352
2,ptyl,ptyl_beta=0.001_lambd=10.0_lr_0=0.1_n=100_rand...,100,3001,1732.840009,0.7138,0.866968
3,ptyl,ptyl_beta=0.001_lambd=10.0_lr_0=0.1_n=1000_ran...,1000,1001,2230.825328,0.9025,0.338868
4,ptyl,ptyl_beta=0.001_lambd=10.0_lr_0=0.1_n=1000_ran...,1000,2001,2175.576511,0.8956,0.363368
5,ptyl,ptyl_beta=0.001_lambd=10.0_lr_0=0.1_n=1000_ran...,1000,3001,2226.462102,0.8947,0.365318
6,ptyl,ptyl_beta=0.01_lambd=1.0_lr_0=0.1_n=10000_rand...,10000,1001,2156.314801,0.9578,0.139635
7,ptyl,ptyl_beta=0.01_lambd=1.0_lr_0=0.1_n=10000_rand...,10000,2001,2214.734112,0.9545,0.155638
8,ptyl,ptyl_beta=0.01_lambd=1.0_lr_0=0.1_n=10000_rand...,10000,3001,2143.781565,0.9556,0.142902
9,ptyl,ptyl_beta=0.01_lambd=1.0_lr_0=0.1_n=50000_rand...,50000,1001,2169.002583,0.9700,0.096795


In [8]:
condition = (tuned_df.n==50000)&(tuned_df.criterion=='ptyl')&(tuned_df.random_state==1001)
temp_df = tuned_df.loc[condition]
total_time = temp_df.runtime.sum()
best_model_name = temp_df.loc[temp_df.val_nll.idxmin(), 'model_name']
total_time += retrained_df.loc[retrained_df.model_name==best_model_name, 'runtime'].item()
average_time = total_time/(temp_df.shape[0]+1)
print(f'Average runtime (sec.): {format_time(average_time)}')
print(f'Total time (sec.): {format_time(total_time)}')

Average runtime (sec.): 0.0 hrs. 37.0 mins. 1.5142816013321863 secs.
Total time (sec.): 148.0 hrs. 43.0 mins. 4.941865921020508 secs.


In [9]:
experiments_directory = '/cluster/tufts/hugheslab/eharve06/data-emphasized-ELBo/experiments/retrained_CIFAR-10_VI'
lr_0s = [0.1, 0.01, 0.001, 0.0001]
criterion = 'linear_probing'
ns = [100, 1000, 10000, 50000]
random_states = [1001, 2001, 3001]

columns = ['criterion', 'model_name', 'n', 'random_state', 'runtime', 'train_loss', 'test_acc', 'test_nll']
results_df = pd.DataFrame(columns=columns)

for lr_0, n, random_state in itertools.product(lr_0s, ns, random_states):
    model_name = f'{criterion}_kappa={23528522/n}_lr_0={lr_0}_n={n}_random_state={random_state}'
    temp_df = pd.read_csv(f'{experiments_directory}/{model_name}.csv')
    row = [criterion, model_name, n, random_state, temp_df['train_sec/epoch'].sum(), temp_df.train_loss.values[-1], temp_df.val_or_test_acc.values[-1], temp_df.val_or_test_nll.values[-1]]
    results_df.loc[len(results_df)] = row

In [10]:
condition = (results_df.n==50000)&(results_df.criterion=='linear_probing')&(results_df.random_state==1001)
temp_df = results_df.loc[condition]
total_time = temp_df.runtime.sum()
average_time = total_time/temp_df.shape[0]
print(f'Average runtime (sec.): {format_time(average_time)}')
print(f'Total time (sec.): {format_time(total_time)}')

Average runtime (sec.): 0.0 hrs. 23.0 mins. 39.41444355249405 secs.
Total time (sec.): 1.0 hrs. 34.0 mins. 37.657774209976196 secs.


In [11]:
experiments_directory = '/cluster/tufts/hugheslab/eharve06/data-emphasized-ELBo/experiments/retrained_CIFAR-10_VI'
lr_0s = [0.1, 0.01, 0.001, 0.0001]
criterion = 'l2-sp'
ns = [100, 1000, 10000, 50000]
random_states = [1001, 2001, 3001]

columns = ['criterion', 'model_name', 'n', 'random_state', 'runtime', 'train_loss', 'test_acc', 'test_nll']
results_df = pd.DataFrame(columns=columns)

for lr_0, n, random_state in itertools.product(lr_0s, ns, random_states):
    model_name = f'{criterion}_kappa={23528522/n}_lr_0={lr_0}_n={n}_random_state={random_state}'
    temp_df = pd.read_csv(f'{experiments_directory}/{model_name}.csv')
    row = [criterion, model_name, n, random_state, temp_df['train_sec/epoch'].sum(), temp_df.train_loss.values[-1], temp_df.val_or_test_acc.values[-1], temp_df.val_or_test_nll.values[-1]]
    results_df.loc[len(results_df)] = row

In [12]:
condition = (results_df.n==50000)&(results_df.criterion=='l2-sp')&(results_df.random_state==1001)
temp_df = results_df.loc[condition]
total_time = temp_df.runtime.sum()
average_time = total_time/temp_df.shape[0]
print(f'Average runtime (sec.): {format_time(average_time)}')
print(f'Total time (sec.): {format_time(total_time)}')

Average runtime (sec.): 0.0 hrs. 35.0 mins. 23.964084446430206 secs.
Total time (sec.): 2.0 hrs. 21.0 mins. 35.856337785720825 secs.


In [13]:
experiments_directory = '/cluster/tufts/hugheslab/eharve06/data-emphasized-ELBo/experiments/retrained_CIFAR-10_VI'
lr_0s = [0.1, 0.01, 0.001, 0.0001]
criterion = 'ptyl'
ns = [100, 1000, 10000, 50000]
random_states = [1001, 2001, 3001]

columns = ['criterion', 'model_name', 'n', 'random_state', 'runtime', 'train_loss', 'test_acc', 'test_nll']
results_df = pd.DataFrame(columns=columns)

for lr_0, n, random_state in itertools.product(lr_0s, ns, random_states):
    model_name = f'{criterion}_kappa={23528522/n}_lr_0={lr_0}_n={n}_random_state={random_state}'
    temp_df = pd.read_csv(f'{experiments_directory}/{model_name}.csv')
    row = [criterion, model_name, n, random_state, temp_df['train_sec/epoch'].sum(), temp_df.train_loss.values[-1], temp_df.val_or_test_acc.values[-1], temp_df.val_or_test_nll.values[-1]]
    results_df.loc[len(results_df)] = row

In [14]:
condition = (results_df.n==50000)&(results_df.criterion=='ptyl')&(results_df.random_state==1001)
temp_df = results_df.loc[condition]
total_time = temp_df.runtime.sum()
average_time = total_time/temp_df.shape[0]
print(f'Average runtime (sec.): {format_time(average_time)}')
print(f'Total time (sec.): {format_time(total_time)}')

Average runtime (sec.): 0.0 hrs. 36.0 mins. 30.452621459960938 secs.
Total time (sec.): 2.0 hrs. 26.0 mins. 1.81048583984375 secs.
